In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
import pandas as pd

In [14]:
df_train = pd.read_csv('/content/drive/MyDrive/Father/camp2021/titanic/train.csv',index_col='PassengerId')
df_test = pd.read_csv('/content/drive/MyDrive/Father/camp2021/titanic/test.csv',index_col='PassengerId')

df_train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [15]:
df_train.isna().any()

Survived    False
Pclass      False
Name        False
Sex         False
Age          True
SibSp       False
Parch       False
Ticket      False
Fare        False
Cabin        True
Embarked     True
dtype: bool

In [16]:
df_train['Age'] = df_train['Age'].fillna(value=df_train['Age'].median())
df_test['Age'] = df_test['Age'].fillna(value=df_train['Age'].median())

In [17]:
def clean_data(df):
    df.drop(columns=['Ticket', 'Cabin', 'Name'], inplace=True)
    df['Fare'] = df['Fare'].fillna(value=df['Fare'].median())

    df['Age'] = df['Age'].astype(int)

    df['Embarked'].fillna(value=df['Embarked'].mode()[0], inplace=True)
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    
    return df
clean_data(df_train)
clean_data(df_test)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,0,34,0,0,7.8292,2
893,3,1,47,1,0,7.0000,0
894,2,0,62,0,0,9.6875,2
895,3,0,27,0,0,8.6625,0
896,3,1,22,1,1,12.2875,0
...,...,...,...,...,...,...,...
1305,3,0,28,0,0,8.0500,0
1306,1,1,39,0,0,108.9000,1
1307,3,0,38,0,0,7.2500,0


In [20]:
X=df_train[['Pclass','Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']]
y=df_train['Survived']
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,0,22,1,0,7.2500,0
2,1,1,38,1,0,71.2833,1
3,3,1,26,0,0,7.9250,0
4,1,1,35,1,0,53.1000,0
5,3,0,35,0,0,8.0500,0
...,...,...,...,...,...,...,...
887,2,0,27,0,0,13.0000,0
888,1,1,19,0,0,30.0000,0
889,3,1,28,1,2,23.4500,0


# DecisionTreeClassifier


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

clf = DecisionTreeClassifier(random_state=0).fit(X_train,y_train)
predicted=clf.predict(X_test)
accuracy = accuracy_score(predicted, y_test)
print("Accuracy:%.2f%%" % (accuracy * 100.0))

Accuracy:77.13%


# **Random Forest**

In [27]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier().fit(X_train,y_train)
predicted=rf_model.predict(X_test)
accuracy = accuracy_score(predicted, y_test)
print("Accuracy:%.2f%%" % (accuracy * 100.0))

Accuracy:82.51%


## **Random Forest with MinMaxScaler**


In [29]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestClassifier().fit(X_train_scaled,y_train)
predicted=rf_model.predict(X_test_scaled)
accuracy = accuracy_score(predicted, y_test)
print("Accuracy:%.2f%%" % (accuracy * 100.0))

Accuracy:83.41%


# GaussianNB


In [28]:
from sklearn.naive_bayes import GaussianNB
gnb= GaussianNB().fit(X_train, y_train)
predicted=gnb.predict(X_test)
accuracy = accuracy_score(predicted, y_test)
print("Accuracy:%.2f%%" % (accuracy * 100.0))

Accuracy:78.03%


# XGBClassifier


In [33]:
from xgboost import XGBClassifier
xgb = XGBClassifier().fit(X_train, y_train)
predicted=xgb.predict(X_test)
accuracy = accuracy_score(predicted, y_test)
print("Accuracy:%.2f%%" % (accuracy * 100.0))

Accuracy:85.20%


In [34]:
from sklearn.model_selection import GridSearchCV
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)
parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}


In [35]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

In [51]:
grid_search.fit(X, y)


Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  40 tasks      | elapsed:    3.0s
[Parallel(n_jobs=10)]: Done 210 tasks      | elapsed:   22.7s
[Parallel(n_jobs=10)]: Done 460 tasks      | elapsed:   57.5s
[Parallel(n_jobs=10)]: Done 810 tasks      | elapsed:  1.8min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  2.3min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=4, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=42, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=10,
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(60, 220, 40)},
        

In [52]:
predicted=grid_search.predict(df_test)
predicted

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [54]:
d = {'PassengerId': df_test.index, 'Survived': predicted}
df = pd.DataFrame(data=d)
df.set_index('PassengerId')

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
...,...
1305,0
1306,1
1307,0


In [55]:
df.to_csv('camp2021.csv', index=False)
